In [ ]:
!pip install transformers torch
!pip install pandas
!pip install sentencepiece
!pip install -U huggingface_hub
!pip install tqdm
from tqdm import tqdm
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM
import transformers, torch
from transformers import LlamaTokenizer,LlamaForCausalLM,GenerationConfig
import pandas as pd


class data:
    def __init__(self):
        pass

    def load_data(self, url):
        try:
            df = pd.read_csv(url)
            return df
        except Exception as e:
            print(f"load data failed:{e}")
            return None

    def clean_data(self,df):
        df = df.replace('\n','').replace('\r',' ')
        #df =''.join(df.split())
        return df.lower()

    def clean_data2(self,df):
        for column in ['description','transcription']:
            df[column] = df[column].astype(str).apply(self.clean_data)
        return df

    def save_data(self,clean_df,url,):
        new_file_name = r'mtsamples_cleaned.csv'
        new_file_path = url.rsplit('\\', 1)[0] + '\\' + new_file_name
        clean_df.to_csv(new_file_path, index=False)

    def create_save(self):
        file_path = r'/root/tf-logs/mtsamples.csv'
        text = self.load_data(file_path)
        clean_text = self.clean_data2(text)
        #self.save_data(clean_text, file_path)
        return clean_text

In [ ]:
import transformers, torch
from transformers import LlamaTokenizer,LlamaForCausalLM,GenerationConfig
import clean

original_data = clean.data()
cleaned_data = original_data.create_save()
torch.cuda.empty_cache()
limited_data = cleaned_data.head(100)
tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")
model = LlamaForCausalLM.from_pretrained(
    "baffo32/decapoda-research-llama-7B-hf",
    load_in_8bit=False,
    torch_dtype=torch.float16
).to('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化列表来保存结果
patient_ages = []
treatment_summaries = []

# 循环遍历数据
for index, row in tqdm(limited_data.iterrows(), total=limited_data.shape[0]):
    text = f"{row['description']} {row['transcription']}"


    age_prompt = f"Question: What is the patient's age based on this text?\n{text}"
    inputs = tokenizer(age_prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    age_summary_ids = model.generate(**inputs, max_length=2000, num_beams=1, length_penalty=2.0, num_return_sequences=1, early_stopping=True)
    age_summary = tokenizer.decode(age_summary_ids[0], skip_special_tokens=True)
    patient_ages.append(age_summary)


    treatment_prompt = f"Summary: What patient treatment plan is contained in this text?\n{text}"
    inputs = tokenizer(treatment_prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    treatment_summary_ids = model.generate(**inputs, max_length=2000, num_beams=1, length_penalty=2.0, num_return_sequences=1, early_stopping=True)
    treatment_summary = tokenizer.decode(treatment_summary_ids[0], skip_special_tokens=True)
    treatment_summaries.append(treatment_summary)

# 将结果保存到CSV文件
results_df = pd.DataFrame({
    'Patient Age': patient_ages,
    'Treatment Plan': treatment_summaries
})
results_df.to_csv('llama1_summary_results_withoutrestriction.csv', index=False)
